##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Olá, vários mundos

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/hello_many_worlds"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este tutorial mostra como uma rede neural clássica pode aprender a corrigir erros de calibração de qubits. Ele apresenta o <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>, um framework do Python para criar, editar e invocar circuitos de Escala Intermediária de Ruído Quântico (NISQ) e demonstra como o Cirq interage com o TensorFlow Quantum.

## Configuração

In [ ]:
!pip install tensorflow==2.7.0

Instale o TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum==0.7.2

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

Agora importe o TensorFlow e as dependências de módulo:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. Noções básicas

### 1.1 Cirq e circuitos quânticos parametrizados

Antes de explorar o TensorFlow Quantum (TFQ), vamos conferir algumas noções básicas do <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>. O Cirq é uma biblioteca Python para computação quântica do Google. Ele pode ser usado para definir circuitos, incluindo portas estáticas e parametrizadas.

O Cirq usa símbolos <a target="_blank" href="https://www.sympy.org" class="external">SymPy</a> para representar parâmetros livres.

In [ ]:
a, b = sympy.symbols('a b')

O código a seguir cria um circuito de dois qubits usando seus parâmetros:

In [ ]:
# Create two qubits
q0, q1 = cirq.GridQubit.rect(1, 2)

# Create a circuit on these qubits using the parameters you created above.
circuit = cirq.Circuit(
    cirq.rx(a).on(q0),
    cirq.ry(b).on(q1), cirq.CNOT(control=q0, target=q1))

SVGCircuit(circuit)

Para avaliar os circuitos, você pode usar a interface `cirq.Simulator`. É possível substituir parâmetros livres em um circuito com números específicos ao passar um objeto `cirq.ParamResolver`. O código a seguir calcula a saída do vetor de estado bruto do seu circuito parametrizado:

In [ ]:
# Calculate a state vector with a=0.5 and b=-0.5.
resolver = cirq.ParamResolver({a: 0.5, b: -0.5})
output_state_vector = cirq.Simulator().simulate(circuit, resolver).final_state_vector
output_state_vector

Os vetores de estado não podem ser acessados diretamente fora da simulação (observe os números complexos na saída acima). Para ser fisicamente realista, você precisa especificar uma medida, que converte um vetor de estado em um número real que computadores clássicos conseguem entender. O Cirq especifica medições usando combinações de <a target="_blank" href="https://en.wikipedia.org/wiki/Pauli_matrices" class="external">operadores de Pauli</a> $\hat{X}$, $\hat{Y}$ e $\hat{Z}$. Para ilustrar, o código a seguir mede $\hat{Z}_0$ e $\frac{1}{2}\hat{Z}_0 + \hat{X}_1$ no vetor de estado que você acabou de simular:

In [ ]:
z0 = cirq.Z(q0)

qubit_map={q0: 0, q1: 1}

z0.expectation_from_state_vector(output_state_vector, qubit_map).real

In [ ]:
z0x1 = 0.5 * z0 + cirq.X(q1)

z0x1.expectation_from_state_vector(output_state_vector, qubit_map).real

### 1.2 Circuitos quânticos como tensores

O TensorFlow Quantum (TFQ) oferece a função `tfq.convert_to_tensor`, que converte objetos do Cirq em tensores. Isso permite que você envie objetos do Cirq a nossas <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/layers">camadas quânticas</a> e <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/get_expectation_op">ops quânticas</a>. A função pode ser chamada em listas ou arrays de circuitos do Cirq e Paulis do Cirq:

In [ ]:
# Rank 1 tensor containing 1 circuit.
circuit_tensor = tfq.convert_to_tensor([circuit])

print(circuit_tensor.shape)
print(circuit_tensor.dtype)

Isso codifica os objetos Cirq como tensores `tf.string` que as operações `tfq` decodificam conforme necessário.

In [ ]:
# Rank 1 tensor containing 2 Pauli operators.
pauli_tensor = tfq.convert_to_tensor([z0, z0x1])
pauli_tensor.shape

### 1.3 Simulação de circuito em lote

O TFQ oferece métodos para computar valores esperados, amostras e vetores de estado. Por enquanto, vamos focar nos *valores esperados*.

A interface de mais alto nível para calcular os valores esperados é a camada `tfq.layers.Expectation`, que é uma `tf.keras.Layer`. Em sua forma mais simples, essa camada equivale à simulação de um circuito parametrizado em vários `cirq.ParamResolvers`. No entanto, o TFQ permite lotes seguindo a semântica TensorFlow, e os circuitos são simulados usando código C++ eficiente.

Crie um lote de valores para substituir os parâmetros `a` e `b`:

In [ ]:
batch_vals = np.array(np.random.uniform(0, 2 * np.pi, (5, 2)), dtype=float)

A execução do circuito em com os valores de parâmetros no Cirq exige um loop:

In [ ]:
cirq_results = []
cirq_simulator = cirq.Simulator()

for vals in batch_vals:
    resolver = cirq.ParamResolver({a: vals[0], b: vals[1]})
    final_state_vector = cirq_simulator.simulate(circuit, resolver).final_state_vector
    cirq_results.append(
        [z0.expectation_from_state_vector(final_state_vector, {
            q0: 0,
            q1: 1
        }).real])

print('cirq batch results: \n {}'.format(np.array(cirq_results)))

A mesma operação é simplificada no TFQ:

In [ ]:
tfq.layers.Expectation()(circuit,
                         symbol_names=[a, b],
                         symbol_values=batch_vals,
                         operators=z0)

## 2. Otimização clássica-quântica híbrida

Agora que você já sabe o básico, vamos usar o TensorFlow Quantum para construir uma *rede neural clássica-quântica híbrida*. Você treinará uma rede neural clássica para controlar um único qubit. O controle será otimizado para preparar corretamente o qubit no estado `0` ou `1`, resolvendo um erro de calibração sistemático simulado. Esta figura mostra a arquitetura:

<img src="./images/nn_control1.png" width="1000">

Mesmo sem uma rede neural, esse é um problema simples de solucionar, mas o tema é semelhante aos problemas de controle quântico reais que você pode resolver usando o TFQ. Ele demonstra um exemplo completo de computação clássica-quântica usando a camada `tfq.layers.ControlledPQC` (Circuito Quântico Parametrizado) dentro de um `tf.keras.Model`.

Para a implementação deste tutorial, essa arquitetura é dividida em três partes:

- *Circuito de entrada* ou *circuito dos pontos de dados*: as primeiras três portas $R$.
- *Circuito controlado*: as outras três portas $R$.
- *Controlador*: a rede neural clássica que configura os parâmetros do circuito controlado.

### 2.1 Definição do circuito controlado

Defina uma rotação de um único bit que possa ser aprendida, conforme indicado na figura acima. Isso corresponderá ao circuito controlado.

In [ ]:
# Parameters that the classical NN will feed values into.
control_params = sympy.symbols('theta_1 theta_2 theta_3')

# Create the parameterized circuit.
qubit = cirq.GridQubit(0, 0)
model_circuit = cirq.Circuit(
    cirq.rz(control_params[0])(qubit),
    cirq.ry(control_params[1])(qubit),
    cirq.rx(control_params[2])(qubit))

SVGCircuit(model_circuit)

### 2.2 Controlador

Agora defina a rede controladora: 

In [ ]:
# The classical neural network layers.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

A partir de um lote de comandos, o controlador gera um lote de sinais de controle para o circuito controlado.

O controlador é inicializado aleatoriamente, então essas saídas ainda não são úteis.

In [ ]:
controller(tf.constant([[0.0],[1.0]])).numpy()

### 2.3 Conecte o controlador ao circuito

Use `tfq` para conectar o controlador ao circuito controlado, como um único `keras.Model`.

Veja o [guia da API funcional do Keras](https://www.tensorflow.org/guide/keras/functional) para saber mais sobre esse estilo de definição do modelo.

Primeiro, defina as entradas do modelo: 

In [ ]:
# This input is the simulated miscalibration that the model will learn to correct.
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.string,
                                name='circuits_input')

# Commands will be either `0` or `1`, specifying the state to set the qubit to.
commands_input = tf.keras.Input(shape=(1,),
                                dtype=tf.dtypes.float32,
                                name='commands_input')


Em seguida, aplique operações a essas entradas, para definir a computação.

In [ ]:
dense_2 = controller(commands_input)

# TFQ layer for classically controlled circuits.
expectation_layer = tfq.layers.ControlledPQC(model_circuit,
                                             # Observe Z
                                             operators = cirq.Z(qubit))
expectation = expectation_layer([circuits_input, dense_2])

Agora, empacote essa computação como um `tf.keras.Model`:

In [ ]:
# The full Keras model is built from our layers.
model = tf.keras.Model(inputs=[circuits_input, commands_input],
                       outputs=expectation)

A arquitetura da rede é indicada pela plotagem do modelo abaixo. Compare essa plotagem de modelo ao diagrama da arquitetura para verificar se está correta.

Observação: talvez seja necessário instalar o pacote `graphviz` no sistema.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

Esse modelo aceita duas entradas: os comandos para o controlador e o circuito de entrada cuja saída o controlador está tentando corrigir. 

### 2.4 Dataset

O modelo tenta gerar o valor de medição correto de $\hat{Z}$ para cada comando. Os comandos e valores corretos estão definidos abaixo.

In [ ]:
# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired Z expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

Esse não é todo o dataset de treinamento para essa tarefa. Cada ponto de dados no dataset também precisa de um circuito de entrada.

### 2.4 Definição do circuito de entrada

O circuito de entrada abaixo define a calibração aleatória incorreta que o modelo aprenderá a corrigir.

In [ ]:
random_rotations = np.random.uniform(0, 2 * np.pi, 3)
noisy_preparation = cirq.Circuit(
  cirq.rx(random_rotations[0])(qubit),
  cirq.ry(random_rotations[1])(qubit),
  cirq.rz(random_rotations[2])(qubit)
)
datapoint_circuits = tfq.convert_to_tensor([
  noisy_preparation
] * 2)  # Make two copied of this circuit

Há duas cópias do circuito, uma para cada ponto de dados.

In [ ]:
datapoint_circuits.shape

### 2.5 Treinamento

Com as entradas definidas, você pode fazer uma execução de teste do modelo `tfq`.

In [ ]:
model([datapoint_circuits, commands]).numpy()

Agora, realize um processo de treinamento padrão para ajustar esses valores em relação a `expected_outputs`.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x=[datapoint_circuits, commands],
                    y=expected_outputs,
                    epochs=30,
                    verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

A partir dessa plotagem, você pode ver que a rede neural aprendeu a resolver a calibração sistemática incorreta.

### 2.6 Verifique as saídas

Agora, use o modelo treinado para corrigir os erros de calibração de qubits. Com o Cirq:

In [ ]:
def check_error(command_values, desired_values):
  """Based on the value in `command_value` see how well you could prepare
  the full circuit to have `desired_value` when taking expectation w.r.t. Z."""
  params_to_prepare_output = controller(command_values).numpy()
  full_circuit = noisy_preparation + model_circuit

  # Test how well you can prepare a state to get expectation the expectation
  # value in `desired_values`
  for index in [0, 1]:
    state = cirq_simulator.simulate(
        full_circuit,
        {s:v for (s,v) in zip(control_params, params_to_prepare_output[index])}
    ).final_state_vector
    expt = cirq.Z(qubit).expectation_from_state_vector(state, {qubit: 0}).real
    print(f'For a desired output (expectation) of {desired_values[index]} with'
          f' noisy preparation, the controller\nnetwork found the following '
          f'values for theta: {params_to_prepare_output[index]}\nWhich gives an'
          f' actual expectation of: {expt}\n')


check_error(commands, expected_outputs)

O valor da função de perda durante o treinamento oferece uma ideia aproximada do desempenho de aprendizado do modelo. Quanto menor for a perda, mais perto os valores esperados na célula acima estarão de `desired_values`. Se você não estiver tão preocupado com os valores dos parâmetros, sempre poderá verificar as saídas de cima usando `tfq`:

In [ ]:
model([datapoint_circuits, commands])

## 3 Como preparar eigenstates de operadores diferentes

A escolha dos eigenstates $\pm \hat{Z}$ correspondentes a 1 e 0 foi arbitrária. Você também pode facilmente querer que 1 corresponda ao eigenstate $+ \hat{Z}$ e 0 ao eigenstate $-\hat{X}$. Uma maneira de conseguir isso é especificando um operador de medição diferente para cada comando, conforme indicado na figura abaixo:

<img src="./images/nn_control2.png" width="1000">

Isso requer o uso de <code>tfq.layers.Expectation</code>. Agora, sua entrada inclui três objetos: circuito, comando e operador. A saída ainda é o valor esperado.

### 3.1 Definição do novo modelo

Vamos observar o modelo para a realização dessa tarefa:

In [ ]:
# Define inputs.
commands_input = tf.keras.layers.Input(shape=(1),
                                       dtype=tf.dtypes.float32,
                                       name='commands_input')
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.dtypes.string,
                                name='circuits_input')
operators_input = tf.keras.Input(shape=(1,),
                                 dtype=tf.dtypes.string,
                                 name='operators_input')

Aqui está a rede controladora:

In [ ]:
# Define classical NN.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

Combine o circuito e o controlador em um único `keras.Model` usando `tfq`:

In [ ]:
dense_2 = controller(commands_input)

# Since you aren't using a PQC or ControlledPQC you must append
# your model circuit onto the datapoint circuit tensor manually.
full_circuit = tfq.layers.AddCircuit()(circuits_input, append=model_circuit)
expectation_output = tfq.layers.Expectation()(full_circuit,
                                              symbol_names=control_params,
                                              symbol_values=dense_2,
                                              operators=operators_input)

# Contruct your Keras model.
two_axis_control_model = tf.keras.Model(
    inputs=[circuits_input, commands_input, operators_input],
    outputs=[expectation_output])

### 3.2 Dataset

Agora você também incluirá os operadores que deseja medir para cada ponto de dados fornecido a `model_circuit`:

In [ ]:
# The operators to measure, for each command.
operator_data = tfq.convert_to_tensor([[cirq.X(qubit)], [cirq.Z(qubit)]])

# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

### 3.3 Treinamento

Agora que você tem novas entradas e saídas, pode treinar novamente usando o keras.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()

two_axis_control_model.compile(optimizer=optimizer, loss=loss)

history = two_axis_control_model.fit(
    x=[datapoint_circuits, commands, operator_data],
    y=expected_outputs,
    epochs=30,
    verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

A função de perda caiu para zero.

O `controller` está disponível como um modelo stand-alone. Chame o controlador e verifique sua resposta ao sinal de cada comando. Seria um pouco trabalhoso comparar corretamente essas saídas ao conteúdo de `random_rotations`.

In [ ]:
controller.predict(np.array([0,1]))

Sucesso: veja se você consegue adaptar a função `check_error` do seu primeiro modelo para funcionar com essa nova arquitetura de modelo.